In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import pandas as pd
import sklearn
import os
import albumentations as A
from PIL import Image
from torchvision.transforms import Resize
import torch

In [ ]:
paths = ['TST_' + str(i + 1).zfill(4) + '.nii.gz' for i in range(421)]
imgs_paths = [os.path.join('test_uncompressed', path) for path in paths]
masks_paths = [os.path.join('test_masks/masks2', path) for path in paths]

In [ ]:
def cut_proj(proj):
    xs, ys, zs = np.where(proj!=0)
    cut_img = proj[min(xs) : max(xs) + 1, min(ys) : max(ys) + 1, min(zs) : max(zs) + 1] 
    return cut_img

In [ ]:
for i in range(0, 421, 20):
    start = i
    finish = i + 20 if i != 400 else 421 
    print(f'batch from {start} to {finish}')
    print('-> Uploading Images')
    imgs = [nib.load(imgs_paths[i]) for i in range(start, finish)]
    print('-> Images Uploaded \n-> Uploading Masks')
    masks = [nib.load(masks_paths[i]) for i in range(start, finish)]
    print('-> Masks Uploaded')
    print('-> Getting data of Images')
    imgs_data = []
    for img in imgs:
        try:
            img = img.get_fdata()
            imgs_data.append(img)
        except OSError:
            print(imgs.index(img))

    print('-> Images Data Uploaded')
    print('-> Getting data of Masks')
    masks_data = [mask.get_fdata() for mask in masks]
    print('-> Masks Data Uploaded')

    projections_3d = [np.multiply(mask_data, img_data) for mask_data, img_data in zip(masks_data, imgs_data)]

    right_lungs = [np.float32(cut_proj(projection[:255,:,:])) for projection in projections_3d] 
    left_lungs = [np.float32(cut_proj(projection[255:,:,:])) for projection in projections_3d] 

    save_images(start, right_lungs, left_lungs)
    print('End of batch')

In [ ]:
# Save Right Lungs
def save_images(start, right_lungs, left_lungs):
    i = 1 + start
    base_path = 'test_images_np/right_lungs/'
    for array in right_lungs:
        np.save(f'{base_path}TST_' + str(i).zfill(4), array)
        i += 1

    # Save Left Lungs
    i = 1 + start
    base_path = 'test_images_np/left_lungs/'
    for array in left_lungs:
        np.save(f'{base_path}TST_' + str(i).zfill(4), array)
        i += 1

## Process images

In [ ]:
def resize_img(img, size):
    resize = Resize(size)
    resized_img = resize(torch.from_numpy(img).T).T
    return resized_img
def average_img(img, axis):
    average = np.average(img, axis = axis) + 1024
    norm_img = (average) / np.amax(average)
    return norm_img

def max_img(img, axis):
    norm_img = (np.amax(img, axis = axis) + 1024) / 1500
    return norm_img

def std_img(img, axis):
    std = (np.std(img, axis = axis) + 1024)
    norm_img = std / np.amax(std)
    return norm_img
def get_2d_proj(images, axis, size = (256, 256)):
    proj_2d = [resize_img(np.dstack(( average_img(img, axis = axis), max_img(img, axis = axis), \
                    std_img(img, axis = axis) )), size) for img in images]
    return proj_2d

In [ ]:

for i in range(0, 401, 20):
    start = i
    finish = i + 20 if i != 400 else 421 
    print(f'Batch from {start + 1} to {finish}')
    print('loading images')
    
    right_lungs = [np.load('test_images_np/right_lungs/TST_' + str(i).zfill(4) + '.npy') for i in range(start+1, 1+finish)]
    left_lungs = [np.load('test_images_np/left_lungs/TST_' + str(i).zfill(4) + '.npy') for i in range(start+1, 1+finish)]
    print('images loaded')
    lungs = []
    for lungs_3d in (right_lungs, left_lungs):
        projections_2d_dim_0 = get_2d_proj(lungs_3d, axis = 0, size = (128, 256))

        projections_2d_dim_1 = get_2d_proj(lungs_3d, axis = 1)

        projections_2d_dim_2 = get_2d_proj(lungs_3d, axis = 2)

        projections_2d = list(zip(projections_2d_dim_0, projections_2d_dim_1, projections_2d_dim_2))
        # projections_2d is a list of lists [#person1# [2D RGB image axis = 0, 2D RGB image axis = 1, 2D RGB image axis = 2], 
                       #person2# [2D RGB image axis = 0, 2D RGB image axis = 1, 2D RGB image axis = 2], ...]
        lungs.append(projections_2d)
    print('saving images')
    save_jpg(lungs, start, finish)
    print('images saved')
# lungs = {'right': lungs[0], 'left': lungs[1]}
# OUTPUT: dict with right and left lungs separated in projections_2d format.  

In [ ]:
def save_jpg(lungs, start, finish):
    base_path = 'test_jpg'
    j=-1
    for side_lungs in lungs:
        j+=1
        side = 'right' if j == 0 else 'left'
        for i in range(finish - start):
            patient_id = i + start + 1  #starting for 1 the num of patients
            patient = side_lungs[i]
            for axis in range(3):
                img = (np.array(side_lungs[i][axis])*255).astype(np.uint8)
                img = Image.fromarray(img, 'RGB')
                img.save(os.path.join(base_path, f'axis_{axis}/TST_' + str(patient_id).zfill(4) + f'_{side}.jpg'))
